# Calibrating the Zahn Cup (Steps 2 and 3)

## Step 2: Taking the Data



We now have an Arrhenius function of the form $\nu = \nu_0 e^{B/T} $ that describes the viscosity of the N35 reference standard at any temperature between $10-50^o C$.  Where do we go from here?

<br>

Let's think about the calibration process in a simpler case, and compare it to what we are doing here.  When we calibrate a load cell, for example, we used known reference standards for force as an input  (masses pulled down by gravity).  The load cell $transduces$ that input into an output: voltage.  And so we created a plot with the input on the x-axis and the output on the y-axis:

<br>

<center>
<img src = https://github.com/MAugspurger/Exper_Eng/raw/main/Labs_and_Sensors/Images/voltage_force.PNG width = 300>
</center>

<br>

When we calibrate the load cell, we fit a curve to the voltage vs. force data points to find the *sensitivity* of the load cell to force, in units $V/N$.

<br>

Now let's think about the current problem.   What is our known input?  It is the quality of the measurement standard that we know: the viscosity.  And what is this input transduced into?  In other words, how does this input become "readable"?   It becomes readable as a time.  So our calibration plot, with input on the x-axis and output on the y-axis, should look like this:

<br>

<center>
<img src = https://github.com/MAugspurger/Exper_Eng/raw/main/Labs_and_Sensors/Images/viscosity_time_dynamic.PNG width = 300>
</center>

<br>

So we want to take data in a way that leads to a plot of this form.  Your tools will be a timer and a thermometer, so you'll have to think about how to use that thermometer to control the input value.

<br>

<center>
<img src = https://github.com/MAugspurger/Exper_Eng/raw/main/Labs_and_Sensors/Images/step_one.PNG width = 300>
</center>

<br>

So go on now!  Take some data!

<br>

Note: For a dependable curve, you'll want to take 8-10 data points at temperatures spread over the range of interest ($10-50^o~C$).

---

## Step 3: Finding the Zahn Cup coefficients


When we calibrate a load cell, we collect data showing voltage as a function of force, and then fit the curve to that data.  Now we have collected data showing time as function of viscosity (in Step 2), and want to find a fitted curve for that data.  

In particular, we need to find the coefficients $K$ and $c$ that are listed in the Zahn cup manuals:

<br>

$$\nu = K(t-c)$$

<br>

Notice that this is a linear equation, but just in a slightly different form.  It is put in this form to indicate that $t$ must be significantly greater than $c$ for a given liquid in order to get usable results.

---

### Entering your Step 2 data

First, make sure that your data is in the following form.  Note that viscosity is on the x-axis, because it is the input into our calibration plot:

<br>

<center>
<img src = https://github.com/MAugspurger/Exper_Eng/raw/main/Labs_and_Sensors/Images/visc_data2.PNG width = 200>
</center>

<br>

PLEASE NOTE: Double-check a couple things that often cause problems:

* Note that in step 2 you recorded time and *temperature*.  You'll need to use the values you found in Step 1 to turn your temperatures into viscosities.
* Make sure that the viscosity values are in numerical order
* The data in the image above is just example data.  Be sure to use your own data from Step 2!

If we had a large data set, we'd want to upload our data directly to Colab.  But with a small data set, it's a bit easier to enter it manually.  Do that in the cell below.

In [ ]:
# NOTE: Replace the data here with your experimental data from Step 2!
# Make sure the viscosity data is in numerical order!
import numpy as np
import pandas as pd

visc = np.array([200.6, 310.0, 330.3])
time = ([32.1, 33.4, 45.2])

# Put the data in a Series
vis_data = pd.Series(data=time,index=visc)

Plot your data and make sure it matches your spreadsheet:

In [ ]:
# Plot your raw data
vis_data.plot(style='o', ylabel='Time (s)', xlabel='Viscosity (cP)');

Your data points should be roughly linear and the slope should be positive (that is, time should increase as viscosity increases).  If you only see 3 data points here and your viscosities range from 200-300 cP, go back to the beginning of this section and read more carefully!

---

### Making a function that creates a linear curve

Now we want to find a linear fit for this data: we want it to be linear because the assumption with a Zahn Cup is that the relationship between viscosity and time of flow is linear.  

<br>

To find this curve, we'll do the same least squares process we did in the step 1:

<br>

1. Define a `linear_zahn_func` that creates a curve of the form $K(t-c)$ and takes the desired coefficients as arguments (like `arrhenius()` in step 1).
2. Define a `deviation_func` that finds the error between a known data set and the calculated curve.
3. Run `leastsq()` to minimize this deviation.

<br>

First, create the `linear_zahn_func`.  Remember that we have a plot of $t$ as a function of $\nu$, so you'll need to rearrange the equation so that it takes an array of $\nu$ values and returns times. Use `arrhenius()` from step 1 as a guide!  The function should return a `Series` of time values:

In [ ]:
# Create a function that creates a line using the coefficients K and c
def linear_zahn_func(K,c,nu_array):


In [ ]:
# Now test your function with a example array of possible viscosities
# We'll use the manufacturer's recommended values for K and c for the #2 cup
nu_array = np.linspace(20,100,81)
example_times = linear_zahn_func(3.5, 14, nu_array)
example_times.plot(xlabel='Viscosity (cSt)', ylabel='Time (s)');

Your plot should be linear, and have time values ranging from 20 to 45 s.

---

### Making a function for the deviation

✅ ✅ Active Learning:  Now write a `deviation_func()` that uses the Zahn function instead of the Arrhenius function.  Use `deviation_func()` from Step 1 as a model.

In [ ]:
def deviation_func(params, known_data):


And test that function out, again with the manufacturer's values:

In [ ]:
params = [3.5, 14]
zahn = linear_zahn_func(params[0],params[1],vis_data.index)
vis_data.plot(ylabel = 'Time(s)', xlabel = 'Viscosity (cSt)', label='Experimental Data',
           style = 'o');
zahn.plot(label='Manufacturers Predicted Times', legend=True)
deviation_func(params,vis_data);

The blue dots are the experimental data, and the orange line are the times predicted by the manufacturer.  You should see that the manufacturer's calibrated numbers are not very accurate.  

<br>

But we can fix that!  We will now find a new $K$ and $c$ that will minimize this difference between the experimental times and the times predicted by the $\nu = K(t-c)$ sensitivity curve.

---

### Optimizing $K$ and $c$

✅ ✅ Active Learning:  Now you can use `leastsq()` to find the best coefficients.  This should not require significant changes from the step 1.

In [ ]:
# Call leastsq() and print out the values of your optimized parameters


And compare the experimental data to the fitted curve:

In [ ]:
# You might need to change variable names, but otherwise no change needed
nu_range = np.linspace(20,200,50)
zahn_best = linear_zahn_func(best_params[0],best_params[1],nu_range)
vis_data.plot(ylabel = 'Time (s)', xlabel = 'Viscosity (cSt)',
           title = 'Time of flow vs. Viscosity', style = 'o',legend=True);
zahn_best.plot(label='Fitted Arrhenius Curve', legend =True);

If all went well, you should see a nicely fitted line!  

---

### Conclusion

✅ ✅ Active Learning:  Create a code cell below this one, and use it to print out the optimized values for the two Zahn cup coefficients, $K$ and $c$.

✅ ✅ Identify the values of the two optimized coefficients in this text cell.  

MAIN POINT: Make a note of these values for $K$ and $c$, as you'll use those when you test for viscosity using the Zahn cup.  These optimized values *replace* the values provided by the manufacturer.  

<br> In doing so, this will make the Zahn cup much more accurate, because we calibrated it in our particular lab space with the Zahn cup that you will actually use.